#### 기본 정보
- 서울대학교병원 (56032명, 154411건 데이터)
- 442*512사이즈 png ~~512\*512 사이즈 png~~
- -score상 골다공증 기준 (-2.5 이하인 경우에 골다공증)
    - L1-L4 T-Score', 'Neck T-Score', 'Total T-Score' 중 가장 낮은 값 기준 

#### Binary Classificaiton 구성
1. csv 전처리
2. 데이터 분할
3. Dataset 생성
4. 모델 구축
5. 학습 및 테스트

#### 고민
- 같은 사람 사진은 하나만 사용해야할까?
- trian 데이터 수 늘려보기

# Binary Classification

In [33]:
from PIL import Image
import os
import random
import matplotlib.pyplot as plt
import pandas as pd

In [34]:
df = pd.read_csv('./data/Train_SNUH_CPA_90d_osteo_2class.csv')

/tmp/ipykernel_208/2037917953.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/Train_SNUH_CPA_90d_osteo_2class.csv')


In [35]:
df

,pid,fp,label,cxr_StudyDate,L1-L4 T-Score,Neck T-Score,Total T-Score,dxa_StudyDate,cxr_dxa_diff,StudyID,PatientBirthDate,PatientSex,PatientSize,PatientWeight,연구별 환자 ID,최종수진일,org_dir,PathToFolder,FileName
0,32764659,10000089_32764659_20090113_1.png,0,2009-01-13,-1.0,-0.5,0.1,2009-01-13,0.0,32764659_2009-01-13 00:00:00,19411220.0,F,1.52,58.4,R-1414-00035092,2017-09-18,part1,10000089_32764659_20090113,1.dcm
1,38248009,10000451_38248009_20090113_1.png,1,2009-01-13,-3.2,0.2,-0.2,2008-11-24,50.0,38248009_2009-01-13 00:00:00,19460806.0,F,1.56,60.9,R-1414-00051293,2016-10-27,part1,10000451_38248009_20090113,1.dcm
2,38522783,10000477_38522783_20090113_1.png,1,2009-01-13,-2.5,-2.6,-2.1,2009-01-12,1.0,38522783_2009-01-13 00:00:00,19550225.0,F,1.55,56.5,R-1414-00052208,2025-04-08,part1,10000477_38522783_20090113,1.dcm
3,32369960,10000753_32369960_20090113_1.png,0,2009-01-13,0.8,0.9,0.8,2009-01-13,0.0,32369960_2009-01-13 00:00:00,19590102.0,F,1.65,63.7,R-1414-00033931,2025-03-31,part1,10000753_32369960_20090113,1.dcm
4,33532709,10000791_33532709_20090113_1.png,0,2009-01-13,-2.4,-2.1,-1.7,2009-03-30,-76.0,33532709_2009-01-13 00:00:00,19430423.0,M,1.60,51.4,R-1414-00037346,2013-09-17,part1,10000791_33532709_20090113,1.dcm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154406,49245327,997333777_49245327_20170530_1.2.840.113619.2.2...,0,2017-05-30,3.7,0.7,1.7,2017-04-18,42.0,49245327_2017-05-30 00:00:00,19420820.0,M,1.62,56.6,R-1414-00079059,2017-09-19,part2,997333777_49245327_20170530,1.2.840.113619.2.203.4.2147483647.1496124728.7...
154407,49245327,997901553_49245327_20170531_1.3.46.670589.26.8...,0,2017-05-31,3.7,0.7,1.7,2017-04-18,43.0,49245327_2017-05-31 00:00:00,19420820.0,M,1.62,56.6,R-1414-00079059,2017-09-19,part2,997901553_49245327_20170531,1.3.46.670589.26.802065.4.20170531.64800.21563...
154408,49245327,998914752_49245327_20170531_1.3.46.670589.26.8...,0,2017-05-31,3.7,0.7,1.7,2017-04-18,43.0,49245327_2017-05-31 00:00:00,19420820.0,M,1.62,56.6,R-1414-00079059,2017-09-19,part2,998914752_49245327_20170531,1.3.46.670589.26.802065.4.20170531.171542.2160...
154409,49415652,999035654_49415652_20170531_1.3.46.670589.26.8...,0,2017-05-31,2.7,1.4,1.2,2017-05-15,16.0,49415652_2017-05-31 00:00:00,19520301.0,F,1.54,79.3,R-1414-00079348,2018-06-27,part2,999035654_49415652_20170531,1.3.46.670589.26.802065.4.20170531.182754.2160...


## 데이터 분할

In [36]:
unique_pids = df['pid'].unique()
print(f'환자 수: {len(unique_pids)}')

환자 수: 56032


In [37]:
from sklearn.model_selection import train_test_split
train_pids, temp_pids = train_test_split(unique_pids, test_size=0.2, random_state=42)
val_pids, test_pids = train_test_split(temp_pids, test_size=0.5, random_state=42)

In [38]:
df_train = df[df['pid'].isin(train_pids)].copy()
df_val = df[df['pid'].isin(val_pids)].copy()
df_test = df[df['pid'].isin(test_pids)].copy()

In [39]:
def describe_split(name, df):
    print(f'{name}')
    print(f'샘플 수: {len(df)}')
    print(f'환자 수: {df['pid'].nunique()}')
    print(f'라벨 분포: {df['label'].value_counts(normalize=True).to_dict()}\n')

describe_split('train', df_train)
describe_split('val', df_val)
describe_split('test', df_test)

train
샘플 수: 123434
환자 수: 44825
라벨 분포: {0: 0.7972033637409466, 1: 0.20279663625905342}

val
샘플 수: 15385
환자 수: 5603
라벨 분포: {0: 0.8085147871303218, 1: 0.19148521286967826}

test
샘플 수: 15592
환자 수: 5604
라벨 분포: {0: 0.8001539250897897, 1: 0.19984607491021036}



In [40]:
df_train.to_csv('data/train.csv', index=False)
df_val.to_csv('data/val.csv', index=False)
df_test.to_csv('data/test.csv', index=False)

## 데이터셋 생성

### 데이터셋 정의

In [41]:
from torch.utils.data import Dataset

#데이터셋 클래스 정의
class CXRDataset(Dataset):
    def __init__(self, dataframe, base_path, transform=None):
        self.df = dataframe.reset_index(drop=True) #어떤 df를 넣어도 문제없도록.
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx): #dataset loader에서 텐서형태로 꺼낼 수 있도록 정의해야 함. 
        row = self.df.iloc[idx]
        img_path = os.path.join(self.base_path, row['fp'])
        image = Image.open(img_path).convert("L")
        if self.transform: #transform.compose()객체를 받으면 해당 작업들이 들어 있음.
            image = self.transform(image)
        return image, int(row['label'])

### 데이터셋 로더

#### transform 정의

In [68]:
from torchvision import transforms

IMG_SIZE = 256

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.5], std = [0.5])
])

#### Dataset 생성

In [69]:
base_path = './data/SNUH_CXR_png/SNUH_CXR_png'

train_dataset = CXRDataset(df_train, base_path, transform=transform)
val_dataset = CXRDataset(df_val, base_path, transform=transform)
test_dataset = CXRDataset(df_test, base_path, transform=transform)

#### DataLoader

In [70]:
from torch.utils.data import DataLoader

BATCH_SIZE = 16 #몇으로 하는게 좋을까

train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=False, pin_memory=True,num_workers=0)

In [72]:
images, labels = next(iter(train_loader))
print('이미지 shape: ', images.shape)
print('라벨 shape:', labels)

이미지 shape:  torch.Size([16, 1, 256, 256])
라벨 shape: tensor([0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])


## 모델 구축

In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    #레이어들을 정의
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1) 

        self.fc1 = nn.Linear(32*64*64, 128) 
        self.fc2 = nn.Linear(128,1)

    #레이어를 쌓아 모델을 정의
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        #print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = x.view(x.size(0), -1)  
        #print(x.shape)
        x = F.relu(self.fc1(x))               
        x = self.fc2(x)                       
        return x

In [74]:
model = SimpleCNN()
print(model)

SimpleCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=131072, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)


## 학습 루프

In [75]:
from sklearn.metrics import roc_auc_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

criterion = torch.nn.BCEWithLogitsLoss() #binary에서 사용? 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 20

for epoch in range(EPOCHS):
    print(f"  🔄 Epoch {epoch+1} 시작")
    model.train()
    train_loss = 0
    y_true, y_pred = [], []

    for images, labels in train_loader:
        #print(f"    📦 배치 불러오기 완료")
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        print(f"    ✅ forward 완료")
        loss = criterion(outputs, labels)
        print(f"    ✅ loss 계산 완료")

        loss.backward()
        optimizer.step()
        print(f"    ✅ backward 완료, optimizer step 완료")

        train_loss += loss.item()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(torch.sigmoid(outputs).cpu().detach().numpy())


    auc = roc_auc_score(y_true, y_pred)
    print(f"[Epoch {epoch+1}] Loss: {train_loss:.4f} | AUROC: {auc:.4f}")


  🔄 Epoch 1 시작
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer step 완료
    ✅ forward 완료
    ✅ loss 계산 완료
    ✅ backward 완료, optimizer

KeyboardInterrupt: 

In [60]:
torch.cuda.is_available()

True

In [66]:
images.device

device(type='cuda', index=0)